This is the workbook to train an NLP model for summarization and keywords identification on Standards of Finance Sustainable practice 


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext

site_url = 'https://yuoffice.sharepoint.com/'
username = 'pelumi@yorku.ca'
password = 'Foodcourt1234'

context_auth = AuthenticationContext(url=site_url)
if context_auth.acquire_token_for_user(username, password):
    ctx = ClientContext(site_url, context_auth)
    web = ctx.web
    ctx.load(web)
    ctx.execute_query()
    print("Web title: {0}".format(web.properties['Title']))
else:
    print(context_auth.get_last_error())

An error occurred while retrieving token from XML response: AADSTS53003: Access has been blocked by Conditional Access policies. The access policy does not allow token issuance.


ValueError: An error occurred while retrieving token from XML response: AADSTS53003: Access has been blocked by Conditional Access policies. The access policy does not allow token issuance.

In [ ]:
#convert pdf to txt file
import PyPDF2

def convert_pdf_to_txt(file_path):
    # Open the PDF file in read-binary mode
    with open(file_path, 'rb') as pdf_file:
        # Create a PDF file reader object
        pdf_reader = PyPDF2.PdfFileReader(pdf_file)

        # Get the number of pages in the PDF
        num_pages = pdf_reader.numPages

        # Loop through each page and extract the text
        text = ''
        for page in range(num_pages):
            text += pdf_reader.getPage(page).extractText()

    # Write the extracted text to a new text file
    with open('output.txt', 'w') as txt_file:
        txt_file.write(text)

# Call the function with the path to your PDF file
convert_pdf_to_txt('path_to_your_pdf.pdf')

In [ ]:
https://yuoffice-my.sharepoint.com/:b:/g/personal/pelumi_yorku_ca/EdU-EDHJj7ZLgeSuSM2G_bEBj8uNJVXvipSljVgyy7Z21w?e=olEVOg